In [1]:
import sys
from pathlib import Path

In [2]:
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

2022-11-30 22:56:49.894833: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-30 22:56:50.064548: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-30 22:56:50.690014: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64
2022-11-30 22:56:50.690086: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: can

In [3]:
import tensorflow_datasets as tfds

/home/mikheil/.conda/envs/ssl-repo/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
sys.path.append(str(Path("../../ssl").resolve()))

In [5]:
from src.models.pi_model.pi_model import PiModel
from src.models.pi_model.pi_model_config import PiModelConfig
from src.trainers.meta_pseudo_label.meta_pseudo_label import MetaPseudoLabelTrainer
from src.trainers.meta_pseudo_label.meta_pseudo_label_config import MetaPseudoLabelTrainerConfig
from src.data_loaders.meta_pseudo_label.meta_pseudo_label_config import MetaPseudoLabelDataLoaderConfig
from src.data_loaders.meta_pseudo_label.meta_pseudo_label import MetaPseudoLabelDataLoader
from src.data_loaders.basic.categorical_ce import CategoricalCEDataLoader
from src.data_loaders.basic.categorical_ce_config import CategoricalCEDataLoaderConfig

# Description

In this notebook, a model will be trained on the CIFAR-10 dataset using only 20% of the labelled dataset. Unlabelled data will be used in training.

## Set up Experiment

In [6]:
class TrainerConfig(MetaPseudoLabelTrainerConfig):
    num_epochs = 175
    num_epochs_finetune = 25

    uda_loss_weight = 1.0
    uda_conf_thresh = 0.80
    uda_softmax_temp = 0.40

train_config = TrainerConfig()

In [7]:
class ModelConfig(PiModelConfig):
    input_shape = (32, 32, 3)
    output_shape = 10
    output_activation = None

model_config = ModelConfig()

In [8]:
class DataLoaderConfigSSL(MetaPseudoLabelDataLoaderConfig):
    batch_size = 64
    num_classes = 10
    shuffle_buffer_size = 50000 # dataset size    

    # apply augmentations
    blur_params = {
        'chance': 0.10,
        'kernel_ratio': 0.10,
        'blur_strength': (0.1, 2.0)
    }

    crop_params = {
        'chance': 0.50,
        'crop_size': (0.08, 1.0),
        'aspect_range': (0.75, 1.33),
        'num_tries': 100
    }

    jitter_params = {
        'chance': 0.50,
        'distort_strength': 0.20,
        'drop_chance': 0.05
    }

data_loader_config_ssl = DataLoaderConfigSSL()

In [9]:
class DataLoaderConfigCE(CategoricalCEDataLoaderConfig):
    batch_size = 64
    num_classes = 10
    shuffle_buffer_size = 5000 # dataset size    

    # apply augmentations
    blur_params = {
        'chance': 0.10,
        'kernel_ratio': 0.10,
        'blur_strength': (0.1, 2.0)
    }

    crop_params = {
        'chance': 0.50,
        'crop_size': (0.08, 1.0),
        'aspect_range': (0.75, 1.33),
        'num_tries': 100
    }

    jitter_params = {
        'chance': 0.50,
        'distort_strength': 0.20,
        'drop_chance': 0.05
    }

data_loader_config_ce = DataLoaderConfigCE()

## Get Datasets

In [10]:
(x_train_full, y_train_full), (x_test_full, y_test_full) = tf.keras.datasets.cifar10.load_data()

In [11]:
# get train dataset by subsampling 20% of the full training dataset (stratified by labels)
# add the rest as unlabelled samples
x_train_unlabelled, x_train_labelled, y_train_unlabelled_, y_train_labelled = train_test_split(
    x_train_full, y_train_full,
    stratify = y_train_full,
    test_size = 0.20, random_state = 42
)

# cast types
y_train_unlabelled = (-1 * np.ones_like(y_train_unlabelled_)).astype(np.int64)
y_train_labelled = y_train_labelled.astype(np.int64)
y_test_full = y_test_full.astype(np.int64)

In [12]:
x_train = np.concatenate((x_train_unlabelled, x_train_labelled), axis = 0)
y_train = np.concatenate((y_train_unlabelled, y_train_labelled), axis = 0)

In [13]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_data = tf.data.Dataset.from_tensor_slices((x_test_full, y_test_full))

2022-11-30 22:56:53.266405: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-30 22:56:53.274064: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-30 22:56:53.274645: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-30 22:56:53.275725: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [14]:
train_data_finetune = tf.data.Dataset.from_tensor_slices((x_train_labelled, y_train_labelled))

In [15]:
# create train dataset
train_data = MetaPseudoLabelDataLoader(train_data, data_loader_config_ssl)(training = True)

In [16]:
# create test dataset
val_data = MetaPseudoLabelDataLoader(val_data, data_loader_config_ssl)(training = False)

In [17]:
# create finetune dataset
train_data_finetune = CategoricalCEDataLoader(train_data_finetune, data_loader_config_ce)(training = True)

In [18]:
print(f"Train dataset size: {train_data.cardinality()}")
print(f"Validation dataset size: {val_data.cardinality()}")
print(f"Train (finetune) dataset size: {train_data_finetune.cardinality()}")

Train dataset size: 782
Validation dataset size: 157
Train (finetune) dataset size: 157


# Train Model

In [19]:
model_student = PiModel(model_config)()
model_teacher = PiModel(model_config)()

In [20]:
trainer = MetaPseudoLabelTrainer(
    model_student,
    model_teacher,
    train_data,
    train_config,
    train_data_finetune,
    val_dataset = val_data)

In [21]:
trainer.train()

2022-11-30 22:56:56.895116: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204


Training loss at epoch 0 is : ltsu 0.85, ltss 8.20, lttu -0.14, ltts 2.36, lttuda 0.86. Validation loss is : 13.32. Validation acc. is : 15.88.
Training loss at epoch 1 is : ltsu 0.69, ltss 8.74, lttu 0.06, ltts 2.20, lttuda 1.04. Validation loss is : 3.38. Validation acc. is : 19.95.
Training loss at epoch 2 is : ltsu 1.24, ltss 4.21, lttu 0.11, ltts 1.99, lttuda 1.17. Validation loss is : 4.04. Validation acc. is : 24.46.
Training loss at epoch 3 is : ltsu 1.15, ltss 4.07, lttu 0.12, ltts 1.90, lttuda 1.12. Validation loss is : 3.89. Validation acc. is : 31.98.
Training loss at epoch 4 is : ltsu 1.10, ltss 4.10, lttu 0.13, ltts 1.85, lttuda 1.08. Validation loss is : 3.90. Validation acc. is : 33.97.
Training loss at epoch 5 is : ltsu 1.09, ltss 3.90, lttu 0.14, ltts 1.79, lttuda 1.05. Validation loss is : 3.50. Validation acc. is : 36.10.
Training loss at epoch 6 is : ltsu 1.12, ltss 3.20, lttu 0.16, ltts 1.73, lttuda 1.04. Validation loss is : 2.84. Validation acc. is : 35.90.
Trai